# numba支持测试
参考 http://numba.pydata.org/numba-doc/dev/reference/numpysupported.html
1. numba擅长加速numpy.ndarray, 因为这是array，不是linkedlist，内存布局良好，而且有很多可以用于编译的信息。
2. numba对numpy的支持有几个方面
    - Universal functions (ufunc) （是一个类型）
        - a function that operates on ndarrays in an element-by-element fashion。 不过据此可以生成类型转换和广播
        - 输入固定长度的输入，提供固定长度的输出
        - 这个类有成员函数和属性。
        - numpy有60种ufunc，可以有accumulate reduce
    - indexing 很快，可以编译为直接内存访问
    - 可以形成ufunc
3. 明确不支持的：任何Python普通对象的出现
4.

In [2]:
import numpy as np
# ufunc
print(np.add.accumulate(np.arange(5)))
np.multiply.reduce(np.arange(5))

[ 0  1  3  6 10]


0

In [1]:
from numba import *


## 首先Python原生语法
参考 http://numba.pydata.org/numba-doc/dev/reference/pysupported.html

In [16]:
class 类:
    静态变量 = [1]
    udlr_luruldrd = np.array([[-1, 0], [1, 0], [0, -1], [0, 1], [-1, -1], [-1, 1], [1, -1], [1, 1]])
    @staticmethod
    @njit
    def 很好(a, b):
        # print(类.静态变量) # 不行
        # return a+b+类.静态变量  # 也不行
        # return a+b+类.udlr_luruldrd  # 也不行
        print(1)
    @staticmethod
    @njit
    def 哈哈(a, b):
        return a+b
类.很好(1,2)
类.哈哈(1,2)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name '类':[0m [1m[1mCannot determine Numba type of <class 'type'>[0m
[1m
File "C:\Users\YeCanming\AppData\Local\Temp\ipykernel_16840\1294352717.py", line 9:[0m
[1m    def 很好(a, b):
        <source elided>
        # return a+b+类.静态变量  # 也不行
[1m        return a+b+类.udlr_luruldrd  # 也不行
[0m        [1m^[0m[0m
[0m

In [8]:
# 全局函数
@njit
def a():
    # b = []
    # print(len(b))  # 空列表不行
    b = [1]
    print(len(b))  # 空列表不行
a()

1


In [11]:
@njit
def test2():
    x = [1, 2, 3]
    # return sum(x) # ok
    # return map(a, x) # not ok
    return min(x) # ok
test2()

1

In [18]:
@njit
def test_tuple():
    x = (1,2)
    # y = tuple([1,2]) # 不支持
    y = (1,2) # 支持
    return x[0] + y[0], x[1]
test_tuple()
# note: 不要随便做类型转换

(2, 2)

In [21]:
@njit
def test_call(a, b):
    a, b = test_tuple()
    return a+b
test_call(1,2)

4

In [44]:
@njit
def test_argwhere():
    def t(x):
        return x%2==0
    a = np.arange(100).reshape((10,10))
    print(a)
    # return np.argwhere(a<=20&t(a))
    return np.argwhere((lambda x:x<=20 & t(x))(a))
    # return np.argwhere(t(a))
    # return np.argwhere(a<=20)


# print(test_argwhere())
a = np.arange(100).reshape((10,10))
(a<=20) & True
a<=20 & True
20&True    # 震惊世界的符号优先级bug

0

In [54]:
@njit(inline='always')
def index1(x,y):
    return x, y
@njit(inline='always')
def index2(x):
    return x[0], x[1]
@njit
def test_indexing():
    x = np.arange(100).reshape(10, 10)
    print(x)
    print(x[2][2])
    print(x[index2([2,2])])
    print(x[index1(2,2)])
    print(x[2,2])
test_indexing()

[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]
 [30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]
 [50 51 52 53 54 55 56 57 58 59]
 [60 61 62 63 64 65 66 67 68 69]
 [70 71 72 73 74 75 76 77 78 79]
 [80 81 82 83 84 85 86 87 88 89]
 [90 91 92 93 94 95 96 97 98 99]]
22
22
22
22


In [56]:
size = 8
@njit
def test_global():
    print(np.arange(size**2).reshape(size, size))
test_global()

[[ 0  1  2  3  4  5  6  7]
 [ 8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23]
 [24 25 26 27 28 29 30 31]
 [32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47]
 [48 49 50 51 52 53 54 55]
 [56 57 58 59 60 61 62 63]]


In [59]:
udlr_luruldrd = np.array([[-1, 0], [1, 0], [0, -1], [0, 1], [-1, -1], [-1, 1], [1, -1], [1, 1]])
@njit
def test_add():
    index = np.argwhere(np.arange(10).reshape(2,5)==3)[0]
    for i in udlr_luruldrd:
        print(i)
        print(i+index)
test_add()

[-1  0]
[-1  3]
[1 0]
[1 3]
[ 0 -1]
[0 2]
[0 1]
[0 4]
[-1 -1]
[-1  2]
[-1  1]
[-1  4]
[ 1 -1]
[1 2]
[1 1]
[1 4]


In [60]:
from numba import jit
@jit()
def create_card_deck():
    values = "23456789TJQKA"
    suites = "CDHS"
    Deck = []
    for x in values:
        for y in suites:
            Deck.append(x + y)
    return Deck

create_card_deck()

['2C',
 '2D',
 '2H',
 '2S',
 '3C',
 '3D',
 '3H',
 '3S',
 '4C',
 '4D',
 '4H',
 '4S',
 '5C',
 '5D',
 '5H',
 '5S',
 '6C',
 '6D',
 '6H',
 '6S',
 '7C',
 '7D',
 '7H',
 '7S',
 '8C',
 '8D',
 '8H',
 '8S',
 '9C',
 '9D',
 '9H',
 '9S',
 'TC',
 'TD',
 'TH',
 'TS',
 'JC',
 'JD',
 'JH',
 'JS',
 'QC',
 'QD',
 'QH',
 'QS',
 'KC',
 'KD',
 'KH',
 'KS',
 'AC',
 'AD',
 'AH',
 'AS']

In [63]:
@njit
def decide():
    candidates = [(1,2), (3,4)]
    max_value, argmax = -np.inf, None
    for action in candidates:
        if action[0]>max_value:
            max_value, argmax = candidates[1], candidates[0]
    return max_value, argmax
decide()

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mCannot unify float64 and UniTuple(int64 x 2) for 'max_value.2', defined at C:\Users\YeCanming\AppData\Local\Temp\ipykernel_16840\2971131936.py (5)
[1m
File "C:\Users\YeCanming\AppData\Local\Temp\ipykernel_16840\2971131936.py", line 5:[0m
[1mdef decide():
    <source elided>
    max_value, argmax = -np.inf, None
[1m    for action in candidates:
[0m    [1m^[0m[0m
[0m
[0m[1mDuring: typing of assignment at C:\Users\YeCanming\AppData\Local\Temp\ipykernel_16840\2971131936.py (5)[0m
[1m
File "C:\Users\YeCanming\AppData\Local\Temp\ipykernel_16840\2971131936.py", line 5:[0m
[1mdef decide():
    <source elided>
    max_value, argmax = -np.inf, None
[1m    for action in candidates:
[0m    [1m^[0m[0m


In [30]:
@njit
def updated(x, y):
    return x+y
@njit
def evaluate(x):
    return ((x-20)**2)
@njit
def test_vectorize(x):
    y = 20
    print(x+y)
    values = evaluate(updated(x, y))
    d = dict()
    for i, a in enumerate(x):
        d[a] = values[i]
    l = x.tolist()
    l.sort(key = d.get)
    print(l[0:100])

x = np.arange(1e7)
test_vectorize(x)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mUnknown attribute 'tolist' of type array(float64, 1d, C)
[1m
File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_1144\3898525276.py", line 15:[0m
[1mdef test_vectorize(x):
    <source elided>
        d[a] = values[i]
[1m    l = x.tolist()
[0m    [1m^[0m[0m
[0m
[0m[1mDuring: typing of get attribute at C:\Users\Administrator\AppData\Local\Temp\ipykernel_1144\3898525276.py (15)[0m
[1m
File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_1144\3898525276.py", line 15:[0m
[1mdef test_vectorize(x):
    <source elided>
        d[a] = values[i]
[1m    l = x.tolist()
[0m    [1m^[0m[0m


In [19]:
def test_hash(x, a=None):
    if a is None:
        a = {}
    return a[x]
x = str(np.arange(100))
# x = [(1,2), (3,4)]
a = {x:120}
test_hash(x, a)

120

In [33]:
@njit
def test_recur(x = 10):
    if x<0:
        return 1
    return test_recur(x-1)+2
test_recur(10)

23

In [ ]:
def test_sort(x):
    x.sort()